# memo

# 1. requirements

In [ ]:
# Datasets
# 事前に下記のデータセットをアップロードしておく
#   faissgpu17
#   pretrained-pytorch-models

In [ ]:
# Common

In [ ]:
# Images
import json
from pathlib import Path
import csv
import numpy as np
import pandas as pd
from PIL import Image
from collections import defaultdict
from tqdm import tqdm
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets.utils import download_url

!pip install --no-index --find-links ../input/faissgpu17 -r ../input/faissgpu17/requirements.txt
import faiss

In [ ]:
# Texts
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from PIL import Image

!pip install --no-index --find-links ../input/faissgpu17 -r ../input/faissgpu17/requirements.txt
import faiss

# 2. conf

In [ ]:
# Common
# dir
INPUT_DATA_PATH_DIR = "/kaggle/input/shopee-product-matching/"
DATA_TYPE = "test" # train or test

In [ ]:
# Images
# dir
MODEL_PATH_IMAGE = "../input/pretrained-pytorch-models/resnet50-19c8e357.pth"

# cpu/gpu
USE_GPU=True
ACCELERATOR="gpu"

# faiss
FEATURE_IMAGES_DIM = 1000 # ベクトルの次元(dimension)

In [ ]:
# Texts
# FEATURE_TEXTS_DIM = 25000
FEATURE_TEXTS_DIM = 25

# 3. utils

# 4. models

In [ ]:
test_df = pd.read_csv(INPUT_DATA_PATH_DIR + DATA_TYPE + '.csv')

In [ ]:
# key: posting_id, value: matches
matches_defaultdict = defaultdict(set)

## 4-1. extract feature images

In [ ]:
# load images
# 画像ファイルの一覧を取得する
# Pathオブジェクトを生成
p = Path(INPUT_DATA_PATH_DIR + DATA_TYPE + "_images/")

# dir直下のファイルとディレクトリを取得
# Path.glob(pattern)はジェネレータを返す。結果を明示するためlist化しているが、普段は不要。
# ファイル名の条件指定
test_images_path_list = list(p.glob("*.jpg"))

In [ ]:
# デバイスを作成する
def get_device(use_gpu):
    if use_gpu and torch.cuda.is_available():
        # これを有効にしないと、計算した勾配が毎回異なり、再現性が担保できない。
        torch.backends.cudnn.deterministic = True
        return torch.device("cuda")
    else:
        return torch.device("cpu")


# デバイスを選択する。
device = get_device(use_gpu=USE_GPU)

In [ ]:
# モデルを作成する
model = torchvision.models.resnet50(pretrained=False)
model.load_state_dict(torch.load(MODEL_PATH_IMAGE))
if torch.cuda.is_available():
    model.cuda()

In [ ]:
# Transforms を作成する
#. (256, 256) にリサイズする
#. 画像の中心に合わせて、(224, 224) で切り抜く
#. RGB チャンネルごとに平均 (0.485, 0.456, 0.406)、分散 (0.229, 0.224, 0.225) で標準化する
transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # 標準化する。
    ]
)

In [ ]:
# 画像ファイルから特徴量を抽出する
feature_images = [] # indexと同じデータ
for test_images_path in tqdm(test_images_path_list):
    # 画像を読み込む（1枚）
    img = Image.open(test_images_path)
    inputs = transform(img)
    inputs = inputs.unsqueeze(0).to(device)
    # 推論する（1枚）
    model.eval()
    outputs = model(inputs).to('cpu').detach().numpy().copy()
    feature_images.append(outputs[0])
    # -> torch.Size([1, 1000]) -> numpy.ndarray

## 4-2. score similarity images

In [ ]:
# faissインデックス作成
faiss_index = faiss.IndexFlatL2(FEATURE_IMAGES_DIM)

In [ ]:
# faissインデックスに追加
faiss_index.add(np.array(feature_images))

In [ ]:
# 近傍探索
distance_similar_images, idx_similar_images = faiss_index.search(np.array(feature_images), 3)

In [ ]:
# idx番号からposting_idを取得して、matches_defaultdictに格納
for i in tqdm(idx_similar_images):
    image_source = test_images_path_list[i[0]].name
    id_source = test_df.query('image == @image_source')['posting_id'].iloc[-1]
    # -> ex. 'test_3588702337'
    
    image_match = test_images_path_list[i[1]].name
    id_match = test_df.query('image == @image_match')['posting_id'].iloc[-1]
    # -> ex. 'test_3588702337 test_4015706929'
    
    matches_defaultdict[id_source].add(id_source)
    matches_defaultdict[id_source].add(id_match)

## 4-3. extract feature texts

In [ ]:
model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = FEATURE_TEXTS_DIM)
feature_texts = model.fit_transform(test_df['title']).toarray()
feature_texts = np.array(feature_texts,dtype=np.float32)

## 4-4. score similarity texts

In [ ]:
# faissインデックス作成
dimension = len(feature_texts[0])
nlist = min(100, len(feature_texts))
quantiser = faiss.IndexFlatL2(dimension) 
faiss_index = faiss.IndexIVFFlat(quantiser, dimension, nlist, faiss.METRIC_L2)

In [ ]:
# faissインデックスの学習・追加
faiss_index.train(feature_texts)
faiss_index.add(feature_texts)

In [ ]:
# 近傍探索
faiss_index.nprobe = 10

s = time.time()
distance_similar_texts, idx_similar_texts = faiss_index.search(feature_texts, 3)
e = time.time()
print("search time: {}".format(e-s))

In [ ]:
test_text_list = list(test_df['title'])

In [ ]:
# idx番号からposting_idを取得して、matches_defaultdictに格納
for i in tqdm(idx_similar_texts):
    text_source = test_text_list[i[0]]
    id_source = test_df.query('title == @text_source')['posting_id'].iloc[-1]
    # -> ex. 'test_3588702337'
    
    text_match = test_text_list[i[1]]
    id_match = test_df.query('title == @text_match')['posting_id'].iloc[-1]
    # -> ex. 'test_3588702337 test_4015706929'
    
    matches_defaultdict[id_source].add(id_source)
    matches_defaultdict[id_source].add(id_match)

## 4-5. score similarity all

# 5. submit

In [ ]:
matches_dict = dict(matches_defaultdict)

In [ ]:
submit_list = []
submit_list.append("posting_id,matches")
for k, v in tqdm(matches_dict.items()):
    v = " ".join(list((v)))
    submit_list.append(k + "," + v)

In [ ]:
with open('submission.csv', 'w') as f:
    for d in submit_list:
        f.write("%s\n" % d)

In [ ]:
!cat submission.csv | head -n 10

# 6. Evaluation

In [ ]:
# # 画像表示
# # id -> 画像ファイル名
# for i in range(1, 10):
#     id_list = submit_list[i].split(",")[1].split(" ")
#     for j in range(len(id_list)):
#         posting_id = id_list[j]
#         image = test_df.query('posting_id == @posting_id')['image'].iloc[-1]

#         #画像の読み込み
#         im = Image.open(INPUT_DATA_PATH_DIR + "train_images/" + image, 'r')

#         #画像をarrayに変換
#         im_list = np.asarray(im)
#         #貼り付け
#         plt.imshow(im_list)
#         #表示
#         print(image)
#         plt.show()
#     print("---")
